In [1]:
import tkinter as tk
from tkinter import filedialog, ttk
from ultralytics import YOLO
import cv2
from PIL import Image, ImageTk
import numpy as np
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import ImageDraw, ImageFont
import matplotlib.pyplot as plt

# Define paths for the YOLO and Faster R-CNN models
YOLO_MODEL_PATHS = {
    "Self Driving": r"C:\Users\abdul\Documents\New folder\Self Driving Car.v3-fixed-small.yolov11\runs\detect\train13\weights\best.pt",
    "UNO": r"C:\Users\abdul\Documents\New folder\Uno Cards.v2-raw.yolov11\runs\detect\train6\weights\best.pt",
}

FASTER_RCNN_MODELS = {
    "UNO-RCNN": {"path": "fasterrcnn_uno.pth", "num_classes": 17},  # UNO Detection
    "Fruits-RCNN": {"path": "fasterrcnn_fruit.pth", "num_classes": 65},  # Fruit Detection
}

# Define the GUI
class ObjectDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Object Detection Interface")
        self.root.geometry("900x700")
        self.root.configure(bg="#2e3f4f")

        # Title Label
        title_label = tk.Label(root, text="Object Detection", font=("Arial", 20, "bold"), bg="#2e3f4f", fg="#ffffff")
        title_label.pack(pady=10)

        # Frame for dropdown and button
        top_frame = tk.Frame(root, bg="#2e3f4f")
        top_frame.pack(pady=10)

        # Dropdown menu for model selection
        self.selected_model = tk.StringVar(value="Self Driving")  # Default selection
        dropdown_label = tk.Label(top_frame, text="Select Model:", font=("Arial", 12), bg="#2e3f4f", fg="#ffffff")
        dropdown_label.grid(row=0, column=0, padx=5)

        self.model_dropdown = ttk.Combobox(
            top_frame,
            textvariable=self.selected_model,
            values=list(YOLO_MODEL_PATHS.keys()) + list(FASTER_RCNN_MODELS.keys()),  # Combine YOLO and RCNN options
            font=("Arial", 12),
            state="readonly",
        )
        self.model_dropdown.grid(row=0, column=1, padx=5)

        # Upload Button
        self.upload_button = ttk.Button(top_frame, text="Upload Image", command=self.upload_image)
        self.upload_button.grid(row=0, column=2, padx=10)

        # Create a canvas to display images
        self.canvas_frame = tk.Frame(root, bg="#2e3f4f")
        self.canvas_frame.pack(pady=10)
        self.canvas = tk.Canvas(self.canvas_frame, width=800, height=550, bg="#ffffff")
        self.canvas.pack()

        # Load Faster R-CNN models into memory
        self.loaded_rcnn_models = self.load_faster_rcnn_models()

    def load_yolo_model(self, model_name):
        # Load the YOLO model based on the selected dataset
        model_path = YOLO_MODEL_PATHS[model_name]
        return YOLO(model_path)

    def load_faster_rcnn_models(self):
        # Load all Faster R-CNN models into a dictionary
        models = {}
        for name, info in FASTER_RCNN_MODELS.items():
            model = fasterrcnn_resnet50_fpn(pretrained=False, num_classes=info["num_classes"])
            model.load_state_dict(torch.load(info["path"], map_location=torch.device("cpu")))
            model.eval()
            models[name] = model
        return models

    def upload_image(self):
        # Open file dialog to select an image
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
        if file_path:
            # Perform object detection
            self.detect_and_display(file_path)

    def detect_and_display(self, image_path):
        selected_model = self.selected_model.get()
        if selected_model in YOLO_MODEL_PATHS:
            # YOLO inference
            model = self.load_yolo_model(selected_model)
            image = cv2.imread(image_path)
            results = model.predict(source=image_path, conf=0.5)
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    class_id = int(box.cls[0].cpu().numpy())
                    confidence = box.conf[0].cpu().numpy()
                    label = f"Class {class_id} ({confidence:.2f})"
                    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        elif selected_model in FASTER_RCNN_MODELS:
            # Faster R-CNN inference
            model = self.loaded_rcnn_models[selected_model]
            image = Image.open(image_path).convert("RGB")
            input_tensor = F.to_tensor(image).unsqueeze(0)
            with torch.no_grad():
                predictions = model(input_tensor)[0]
            draw = ImageDraw.Draw(image)
            for idx, box in enumerate(predictions["boxes"]):
                if predictions["scores"][idx] >= 0.5:
                    xmin, ymin, xmax, ymax = box.tolist()
                    label = f"{int(predictions['labels'][idx])} ({predictions['scores'][idx]:.2f})"
                    draw.rectangle([xmin, ymin, xmax, ymax], outline="red", width=2)
                    draw.text((xmin, ymin), label, fill="red")
            image = np.array(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Convert image to RGB and display
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(image_rgb)
        tk_image = ImageTk.PhotoImage(pil_image)

        self.canvas.create_image(0, 0, anchor=tk.NW, image=tk_image)
        self.canvas.image = tk_image

# Run the application
root = tk.Tk()
app = ObjectDetectionApp(root)
root.mainloop()


c:\Users\abdul\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\abdul\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
